In [ ]:
import pandas as pd
import random

In [ ]:
df = pd.read_csv('analytical-base-table.csv')

In [ ]:
POPULATION_SIZE = 100
N_GENERATIONS = 100
MUTATION_PROBABILITY = 0.5

In [ ]:
def fitness(individuo, df):
    total = 0
    for pet_idx, adot_idx in enumerate(individuo):
        linha = df[
            (df.pet_index == pet_idx) & 
            (df.adotante_index == adot_idx)
        ].iloc[0]
        
        score = 100
        
        # Tipo de animal x preferência
        if linha['prefere_tipo'] != 'indiferente' and linha['tipo'] != linha['prefere_tipo']:
            score -= 20

        # Porte x preferência
        if linha['prefere_porte'] != 'indiferente' and linha['porte'] != linha['prefere_porte']:
            score -= 15

        # Porte grande em apartamento
        if linha['porte'] == 'grande' and linha['moradia'] == 'apartamento':
            score -= 30

        # Pelagem longa com alergia
        if linha['pelagem'] == 'longa' and linha['tem_alergia'] == 1:
            score -= 50

        # Idade muito baixa com pouco tempo disponível
        if linha['idade_meses'] < 6 and linha['tempo_disponivel_horas'] < 2:
            score -= 20

        # Vacinação, vermifugação, esterilização
        if linha['vacinado'] == 0:
            score -= 10
        if linha['vermifugado'] == 0:
            score -= 10
        if linha['esterilizado'] == 0:
            score -= 5

        # Saúde do pet
        if linha['saude'] in ['lesao_leve', 'lesao_grave']:
            score -= 30

        # Criança + pet com problema de saúde
        if linha['tem_crianca'] == 1 and linha['saude'] in ['lesao_leve', 'lesao_grave']:
            score -= 20

        # Presença de outros pets (positivo!)
        if linha['outros_pets'] == 1:
            score += 10

        # Mídia: fotos e vídeos contam pontos positivos
        score += int(linha['qtd_fotos']) * 1
        score += int(linha['qtd_videos']) * 2

        total += max(score, 0)
    return total

In [ ]:
def crossover(pai1, pai2):
    ponto = random.randint(1, len(pai1) - 2)
    filho = pai1[:ponto] + pai2[ponto:]
    return filho

In [ ]:
def mutacao(individuo, num_adotantes, taxa_mut=MUTATION_PROBABILITY):
    for i in range(len(individuo)):
        if random.random() < taxa_mut:
            individuo[i] = random.randint(0, num_adotantes - 1)
    return individuo

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import random
import math

def run_ga(df, num_pets, num_adotantes, pop_size=50, generations=80):
    pop = [[random.randint(0, num_adotantes - 1) for _ in range(num_pets)]
           for _ in range(pop_size)]
    
    history = {'best': [], 'mean': []}

    pets_per_col = 20
    pet_cols = math.ceil(num_pets / pets_per_col)
    spacing_x_pet = 1.5
    spacing_y_pet = 1.0

    pet_positions = []
    for i in range(num_pets):
        col = i // pets_per_col
        row = i % pets_per_col
        x = col * spacing_x_pet
        y = 20 - row * spacing_y_pet
        pet_positions.append((x, y))

    spacing_y_ad = 1.0
    adotante_positions = []
    adotante_x = pet_cols * spacing_x_pet + 5
    for i in range(num_adotantes):
        y = 20 - i * spacing_y_ad
        adotante_positions.append((adotante_x, y))

    fig, ax = plt.subplots(figsize=(12, 12))
    
    for gen in range(generations + 1):
        scores = [fitness(ind, df) for ind in pop]
        sorted_pop = [x for _, x in sorted(zip(scores, pop), key=lambda x: x[0], reverse=True)]
        elites = sorted_pop[:int(0.2 * pop_size)]
        new_pop = elites.copy()

        while len(new_pop) < pop_size:
            p1, p2 = random.sample(elites, 2)
            filho = crossover(p1, p2)
            filho = mutacao(filho, num_adotantes)
            new_pop.append(filho)

        pop = new_pop
        best = max(scores)
        mean = sum(scores) / pop_size
        history['best'].append(best)
        history['mean'].append(mean)
            
        ax.clear()
        ax.set_title(f'Matching — Geração {gen}/{generations} — Melhor Fitness: {best:.1f}', fontsize=14, weight='bold')
        ax.set_xlim(-1, adotante_x + 3)
        ax.set_ylim(-2, 22)
        ax.axis('off')

        ax.text(spacing_x_pet * (pet_cols / 2), 21, f'PETS ({num_pets})', ha='center', fontsize=12)
        ax.text(adotante_x, 21, f'ADOTANTES ({num_adotantes})', ha='center', fontsize=12)
        ax.text(0, -(spacing_y_pet), 'Cores: Vermelho = ruim |  Verde = bom | Azul = top 3 melhores matches', fontsize=10)

        for idx, (x, y) in enumerate(pet_positions):
            ax.plot(x, y, 'bo', markersize=4)

        for idx, (x, y) in enumerate(adotante_positions):
            ax.plot(x, y, 'go', markersize=6)

        match_scores = []
        melhor = sorted_pop[0]
        for pet_idx, adot_idx in enumerate(melhor):
            linha = df[
                (df.pet_index == pet_idx) &
                (df.adotante_index == adot_idx)
            ].iloc[0]
            score = 100
            if linha['prefere_tipo'] != 'indiferente' and linha['tipo'] != linha['prefere_tipo']:
                score -= 20
            if linha['prefere_porte'] != 'indiferente' and linha['porte'] != linha['prefere_porte']:
                score -= 15
            if linha['pelagem'] == 'longa' and linha['tem_alergia'] == 1:
                score -= 50
            score = max(score, 0)
            match_scores.append((score, pet_idx, adot_idx))

        match_scores.sort(reverse=True)
        top_3 = match_scores[:3]

        for score, pet_idx, adot_idx in match_scores:
            x1, y1 = pet_positions[pet_idx]
            x2, y2 = adotante_positions[adot_idx]
            score_norm = score / 100
            if (score, pet_idx, adot_idx) in top_3:
                cor = 'blue'
                lw = 5
                alpha = 1.0
            else:
                cor = (1 - score_norm, score_norm, 0) 
                lw = 1 + score_norm * 3
                alpha = 0.4 + 0.5 * score_norm
        
            ax.plot([x1, x2], [y1, y2], color=cor, linewidth=lw, alpha=alpha)

            xm = (x1 + x2) / 2
            ym = (y1 + y2) / 2
            ax.text(xm, ym, f'{score:.0f}', fontsize=7, color='black', ha='center', va='center')

        clear_output(wait=True)
        display(fig)

    plt.ioff()
    plt.show()
    return sorted_pop[0], history

In [ ]:
num_pets = df['pet_index'].nunique()
num_adotantes = df['adotante_index'].nunique()

In [ ]:
melhor, hist = run_ga(df, num_pets, num_adotantes, pop_size=POPULATION_SIZE, generations=N_GENERATIONS)